In [1]:
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

tf.__version__

'2.6.0'

In [3]:
# !tf2/bin/pip3 install malaya-speech --no-deps
# !tf2/bin/pip3 install python-speech-features tqdm unidecode malaya-boilerplate librosa herpetologist

In [4]:
import malaya_speech
import numpy as np

/home/husein/tf2/lib/python3.6/site-packages/malaya_boilerplate/frozen_graph.py:29: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [5]:
m = hub.KerasLayer('https://tfhub.dev/google/nonsemantic-speech-benchmark/trillsson1/1',
                  trainable=False)

In [6]:
import json
import pickle

with open('/home/husein/youtube/voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [7]:
with open('/home/husein/youtube/voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [8]:
y, _ = malaya_speech.load(sample_files[0])

In [9]:
%%time

m(np.expand_dims(y, 0))['embedding'].numpy()[0]

CPU times: user 7.46 s, sys: 987 ms, total: 8.45 s
Wall time: 942 ms


array([-0.7479081 , -0.05975108, -0.13241395, ..., -0.8564563 ,
        0.6185336 ,  0.05854531], dtype=float32)

In [10]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))  

In [11]:
from tqdm import tqdm

vectors = {}
for f in tqdm(unique_files):
    y, _ = malaya_speech.load(f)
    v = m(np.expand_dims(y, 0))['embedding'].numpy()[0]
    vectors[f] = v

 23%|██▎       | 8239/36237 [2:22:01<3:45:59,  2.06it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 42%|████▏     | 15381/36237 [4:24:35<4:14:32,  1.37it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 60%|█████▉    | 21581/36237 [6:09:56<2:35:43,  1.57it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable


In [12]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    ls.append(labels[i][0])
    scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|██████████| 5900000/5900000 [00:57<00:00, 101815.35it/s]


In [13]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [14]:
calculate_eer(ls, scores)

(0.38045999999999996, array(9320.70703125))